In [1]:
import glfw
import numpy as np
from math import sin, cos
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
from OpenGL.GLU import *
from OpenGL.GLUT import *
from plyfile import PlyData, PlyElement
import pyrr
import math
from PIL import Image
from camera import Camera
import random

In [2]:
vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec3 a_color;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;

out vec3 v_color;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    v_color = a_color;
}
"""

fragment_src = """
# version 330

in vec3 v_color;
out vec4 out_color;

void main()
{
    out_color = vec4(v_color, 1.0);
}
"""

In [4]:
obj_vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec3 normal;;
layout(location = 2) in vec2 a_texture;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;


out vec3 FragPos;
out vec2 v_texture;
out vec3 Normal;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    FragPos = vec3(model * vec4(a_position, 1.0));
    Normal=normal;
    v_texture = a_texture;
}
"""
obj_fragment_src = """
# version 330

in vec3 FragPos;
in vec2 v_texture;
in vec3 Normal;

out vec4 out_color;

uniform sampler2D s_texture;
uniform mat4 view;

void main()
{
    vec4 lightColor = vec4(3.0, 1.5, 0, 1);
    
    //1、设置一个很小的常量环境因子，来假设场景中一些发散的光
    float ambientStrength = 0.1;
    //2、环境光 = 常量环境因子 * 光的颜色
    vec3 ambient =  vec3(ambientStrength * lightColor);
    
    //1、标准化法向量
    vec3 norm = normalize(Normal);
    // 2、标准化定向光线向量
    vec3 lightPos = vec3(20,20,20);
    vec3 lightDir = normalize(lightPos - FragPos);
    // 3、将两者进行点乘，计算出光源对物体影响的大小，由此确定物体的最终颜色
    float diff = max(dot(norm, lightDir), 0.0);
    vec3 diffuse = vec3(diff * lightColor);    
    
    // 1、设置一个合适的镜面光分量
    float specularStrength = 0.5;
    // 2、计算视线方向向量
    vec3 viewDir = normalize(vec3(view) - FragPos);
    // 3、计算反射向量
    vec3 reflectDir = reflect(-lightDir, norm);
    // 4、将两者进行点乘，计算出镜面光影响的大小
    float spec = pow(max(dot(viewDir, reflectDir), 0.0), 32);
    // 5、计算镜面分量
    vec3 specular = vec3(specularStrength * spec * lightColor);
    
    vec3 allLight = ambient + diffuse + specular;
    out_color = vec4(allLight, 1.0) * texture(s_texture, v_texture);
    //out_color = texture(s_texture, v_texture); // * vec4(v_color, 1.0f);
}
"""


In [5]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/male_face_c10s10e5.ply', 'rb') as f:
    face = PlyData.read(f)
#顶点 x y z 色彩 R G B A 数值为1
face_vertices=[]
vertices_xyz=[]
RGB_color=[]
for i in range(len(face.elements[0])):
    vertices_xyz.append(face.elements[0].data[i][0])
    vertices_xyz.append(face.elements[0].data[i][1])
    vertices_xyz.append(face.elements[0].data[i][2])
    RGB_color.append(face.elements[0].data[i][3])
    RGB_color.append(face.elements[0].data[i][4])
    RGB_color.append(face.elements[0].data[i][5])
vertices_xyz = np.array(vertices_xyz, dtype=np.float32)    
RGB_color = np.array(RGB_color, dtype=np.float32)
#顶点归一化及中心化
Min = np.min(vertices_xyz)
Max = np.max(vertices_xyz)
vertices_xyz = (vertices_xyz - Min) / (Max - Min)
Mean = np.mean(vertices_xyz)
vertices_xyz = vertices_xyz-Mean
#色彩归一化
Min = np.min(RGB_color)
Max = np.max(RGB_color)
RGB_color = (RGB_color-Min) / (Max - Min)
for i in range(len(face.elements[0])):
    face_vertices.append(vertices_xyz[0+i*3])
    face_vertices.append(vertices_xyz[1+i*3])
    face_vertices.append(vertices_xyz[2+i*3])
    face_vertices.append(RGB_color[0+i*3])
    face_vertices.append(RGB_color[1+i*3])
    face_vertices.append(RGB_color[2+i*3])
face_vertices = np.array(face_vertices, dtype=np.float32)
#顶点索引，三点一组
face_indices=[]
for i in range(len(face.elements[1])):
    face_indices.append(face.elements[1].data[i][0][0])
    face_indices.append(face.elements[1].data[i][0][1])
    face_indices.append(face.elements[1].data[i][0][2])
face_indices = np.array(face_indices, dtype=np.uint32)

In [6]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/apple.ply', 'rb') as s:
    apple = PlyData.read(s)
#顶点 x y z 色彩 R G B  取颜色为蓝色 0 170 255 归一化 0 0.67 1
apple_vertices = []
obj_size=5 #设定球体大小
for i in range(len(apple.elements[0])):
    apple_vertices.append(apple.elements[0].data[i][0]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][1]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][2]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][3]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][4]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][5]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][6]*obj_size)
    apple_vertices.append(apple.elements[0].data[i][7]*obj_size)
obj_vertices = np.array(apple_vertices, dtype=np.float32)
#顶点索引，三点一组
apple_indices = []
for i in range(len(apple.elements[1])):
    apple_indices.append(apple.elements[1].data[i][0][0])
    apple_indices.append(apple.elements[1].data[i][0][1])
    apple_indices.append(apple.elements[1].data[i][0][2])
    apple_indices.append(apple.elements[1].data[i][0][3])
obj_indices = np.array(apple_indices, dtype=np.uint32)

In [7]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/sphere_without_color.ply', 'rb') as s:
    sphere = PlyData.read(s)
#物体随机坐标
coordinates = []
obj_size=1 #设定球体大小
for i in range(len(sphere.elements[0])):
    pos=[]
    pos.append(sphere.elements[0].data[i][0]*obj_size )
    pos.append(sphere.elements[0].data[i][1]*obj_size)
    pos.append(sphere.elements[0].data[i][2]*obj_size)
    coordinates.append(pos)
coordinates = np.array(coordinates, dtype=np.float32)

In [8]:
cam = Camera()
WIDTH, HEIGHT = 768, 768
lastX, lastY = WIDTH / 2, HEIGHT / 2
first_mouse = True
left, right, forward, backward = False, False, False, False

# the keyboard input callback
def key_input_clb(window, key, scancode, action, mode):
    global left, right, forward, backward
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    if key == glfw.KEY_W and action == glfw.PRESS:
        forward = True
    elif key == glfw.KEY_W and action == glfw.RELEASE:
        forward = False
    if key == glfw.KEY_S and action == glfw.PRESS:
        backward = True
    elif key == glfw.KEY_S and action == glfw.RELEASE:
        backward = False
    if key == glfw.KEY_A and action == glfw.PRESS:
        left = True
    elif key == glfw.KEY_A and action == glfw.RELEASE:
        left = False
    if key == glfw.KEY_D and action == glfw.PRESS:
        right = True
    elif key == glfw.KEY_D and action == glfw.RELEASE:
        right = False
    # if key in [glfw.KEY_W, glfw.KEY_S, glfw.KEY_D, glfw.KEY_A] and action == glfw.RELEASE:
    # left, right, forward, backward = False, False, False, False

# do the movement, call this function in the main loop
def do_movement():
    if left:
        cam.process_keyboard("LEFT", 0.005)
    if right:
        cam.process_keyboard("RIGHT", 0.005)
    if forward:
        cam.process_keyboard("FORWARD", 0.005)
    if backward:
        cam.process_keyboard("BACKWARD", 0.005)

# the mouse position callback function
def mouse_look_clb(window, xpos, ypos):
    global first_mouse, lastX, lastY
    if first_mouse:
        lastX = xpos
        lastY = ypos
        first_mouse = False
    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos
    cam.process_mouse_movement(xoffset, yoffset)
    
#the mouse click callback function
def mouse_button_click(window, button, action, mods):
    if button == glfw.MOUSE_BUTTON_LEFT:
        data = glReadPixels (0, 0, WIDTH, HEIGHT, GL_RGB,  GL_UNSIGNED_BYTE)
        image = Image.new ("RGB", (WIDTH, HEIGHT), (0, 0, 0))
        image.frombytes (data)
        image = image.crop ((0,0,WIDTH, HEIGHT))
        image = image.transpose(Image.FLIP_TOP_BOTTOM)
        image.save ('saveImage.png')
        
#random pick coordinates
def rnd_sphere_coordinate(coordinate):
    return coordinate[random.randint(0,len(coordinate)-1)]

In [9]:
def window_resize(window, width, height):
    glViewport(0, 0, width, height)
    
# initializing glfw library
if not glfw.init():
    raise Exception("glfw can not be initialized!")
# creating the window
window = glfw.create_window(WIDTH, HEIGHT, "My OpenGL window", None, None)
# check if window was created
if not window:
    glfw.terminate()
    raise Exception("glfw window can not be created!")
# set window's position
glfw.set_window_pos(window, 200, 200)
# set the callback function for window resize
glfw.set_window_size_callback(window, window_resize)
# make the context current
glfw.make_context_current(window)

#设置摄像头移动调用
# set the mouse position callback
glfw.set_cursor_pos_callback(window, mouse_look_clb)
# set the keyboard input callback
glfw.set_key_callback(window, key_input_clb)
# capture the mouse cursor
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
# capture the mouse click
glfw.set_mouse_button_callback(window,mouse_button_click)

#initial face shader
face_shader = compileProgram(compileShader(vertex_src, GL_VERTEX_SHADER),compileShader(fragment_src, GL_FRAGMENT_SHADER))
glUseProgram(face_shader)
face_proj_loc = glGetUniformLocation(face_shader, "projection")
face_loc = glGetUniformLocation(face_shader, "model")
face_view_loc = glGetUniformLocation(face_shader, "view")
projection = pyrr.matrix44.create_perspective_projection_matrix(100, 1024/1024, 0.1, 100)#第一个值远近大小，值越大物体越小
glUniformMatrix4fv(face_proj_loc, 1, GL_FALSE, projection)  
   
#initial object shader
obj_shader = compileProgram(compileShader(obj_vertex_src, GL_VERTEX_SHADER), compileShader(obj_fragment_src, GL_FRAGMENT_SHADER))
glUseProgram(obj_shader)
obj_proj_loc = glGetUniformLocation(obj_shader, "projection")
obj_loc = glGetUniformLocation(obj_shader, "model")
obj_view_loc = glGetUniformLocation(obj_shader, "view")  
projection = pyrr.matrix44.create_perspective_projection_matrix(100, 1024/1024, 0.1, 100)#第一个值远近大小，值越大物体越小
glUniformMatrix4fv(obj_proj_loc, 1, GL_FALSE, projection)  
# Vertex Buffer Object 为物体分配顶点对象缓存 存储所有顶点信息，但不包含索引信息。VAO中包含VBO和EBO
VBO = glGenBuffers(2)
# Element Buffer Object 为物体索引信息分配缓存 只存储索引信息每个物体一个缓存
EBO = glGenBuffers(2)

glEnable(GL_DEPTH_TEST) #图像深度三维化
glClearColor(0.68, 0.68, 0.68, 1)#设置背景色

# the main application loop
while not glfw.window_should_close(window):
    glfw.poll_events()
    
    do_movement()
    
    #物体随机坐标
    xyz = rnd_sphere_coordinate(coordinates)
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    #绑定照相机
    view = cam.get_view_matrix()
    glUseProgram(face_shader)
    glUniformMatrix4fv(face_view_loc, 1, GL_FALSE, view)
    glUseProgram(obj_shader)
    glUniformMatrix4fv(obj_view_loc, 1, GL_FALSE, view)
    
    
    #载入面孔
    glUseProgram(face_shader)
    glBindBuffer(GL_ARRAY_BUFFER, VBO[0])
    glBufferData(GL_ARRAY_BUFFER, face_vertices.nbytes, face_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[0])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,face_indices.nbytes, face_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable color
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    
    face_pos =  pyrr.matrix44.create_from_translation(pyrr.Vector3([0, 0, 0])) #定义物体世界坐标 0, 1, -3
    glUniformMatrix4fv(face_loc, 1, GL_FALSE, face_pos)
    glDrawElements(GL_TRIANGLES, len(face_indices), GL_UNSIGNED_INT, None)
    
    #载入球体
    glUseProgram(obj_shader)
    glBindBuffer(GL_ARRAY_BUFFER, VBO[1])
    glBufferData(GL_ARRAY_BUFFER, obj_vertices.nbytes, obj_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[1])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,obj_indices.nbytes, obj_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, obj_vertices.itemsize * 8, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable nx ny nz 
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, obj_vertices.itemsize * 8, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    #enable texture 
    glEnableVertexAttribArray(2)
    glVertexAttribPointer(2, 2, GL_FLOAT, GL_FALSE, obj_vertices.itemsize * 8, ctypes.c_void_p(24))
    
    texture = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, texture)
    # Set the texture wrapping parameters
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    # Set texture filtering parameters
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)

    # load image
    image = Image.open("textures/cat.png")
    image = image.transpose(Image.FLIP_TOP_BOTTOM)
    img_data = image.convert("RGBA").tobytes()
    # img_data = np.array(image.getdata(), np.uint8) # second way of getting the raw image data
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, image.width, image.height, 0, GL_RGBA, GL_UNSIGNED_BYTE, img_data)
    
    obj_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([1,1,1])) #设定物体位置
    glUniformMatrix4fv(obj_loc, 1, GL_FALSE, obj_pos)

    glDrawElements(GL_QUADS, len(obj_indices), GL_UNSIGNED_INT, None)
    
    glfw.swap_buffers(window)
# terminate glfw, free up allocated resources
glfw.terminate()

C:\Users\alpha02\AppData\Local\Temp\ipykernel_5900\2789018283.py:118: DeprecationWarning: FLIP_TOP_BOTTOM is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_TOP_BOTTOM instead.
  image = image.transpose(Image.FLIP_TOP_BOTTOM)
